In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, IterableDataset
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.amp import autocast, GradScaler
from accelerate import Accelerator

In [15]:
from core import data_manager
from backend.MMM import PricePredictorModel
from backend.Dataset import DatasetTimeseries

In [ ]:
class OptimizedTrainer:
    def __init__(self, config):
        accelerator = Accelerator()
        self.device = torch.device(accelerator.device)
        self.config = config
        self.scaler = GradScaler()
        self.amp = config.get('mixed_precision', True)
        
        # Инициализация модели с оптимизированными параметрами
        self.model = PricePredictorModel().to(self.device)
        self.model = torch.compile(self.model)  # Pytorch 2.0 compilation
        
    def _get_dataloader(self, generator):
        dataset = OptimizedTimeSeriesDataset(
            generator,
            seq_len=self.config['seq_len'],
            pred_len=self.config['pred_len'],
            batch_size=self.config['batch_size']
        )
        
        return DataLoader(
            dataset,
            batch_size=None,  # Уже батчируется в Dataset
            pin_memory=False,
            num_workers=self.config['num_workers'],
            persistent_workers=True,
            prefetch_factor=2
        )

    def _train_epoch(self, train_loader, optimizer):
        self.model.train()
        total_loss = 0.0
        
        for x, y, time_x in train_loader:
            x = x.to(self.device, non_blocking=True)
            y = y.to(self.device, non_blocking=True)
            time_x = time_x.to(self.device, non_blocking=True)
            
            optimizer.zero_grad(set_to_none=True)
            
            with autocast(enabled=self.amp):
                preds = self.model(x, time_x)
                loss = vectorized_quantile_loss(preds, y)
            
            self.scaler.scale(loss).backward()
            self.scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.scaler.step(optimizer)
            self.scaler.update()
            
            total_loss += loss.item() * x.size(0)
        
        return total_loss / len(train_loader.dataset)

    def train(self):
        train_gen, val_gen = self.data_manager.get_generators()
        
        train_loader = self._get_dataloader(train_gen)
        val_loader = self._get_dataloader(val_gen)

        optimizer = torch.optim.AdamW(
            self.model.parameters(),
            lr=self.config['lr'],
            weight_decay=self.config['weight_decay'],
            fused=True  # Использование fused implementation
        )
        
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer,
            max_lr=self.config['lr'],
            total_steps=self.config['epochs'] * len(train_loader),
            pct_start=0.3
        )

        best_loss = float('inf')
        patience_counter = 0

        for epoch in range(self.config['epochs']):
            train_loss = self._train_epoch(train_loader, optimizer)
            val_loss = self._evaluate(val_loader)
            
            # Обновление планировщика
            scheduler.step()
            
            # Логирование и early stopping
            if val_loss < best_loss:
                best_loss = val_loss
                patience_counter = 0
                torch.save(self.model.state_dict(), "best_model.pth")
            else:
                patience_counter += 1
                if patience_counter >= self.config['patience']:
                    print("Early stopping triggered")
                    break

            print(f"Epoch {epoch+1:03d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

    @torch.no_grad()
    def _evaluate(self, dataloader):
        self.model.eval()
        total_loss = 0.0
        
        for x, y, time_x in dataloader:
            x = x.to(self.device, non_blocking=True)
            y = y.to(self.device, non_blocking=True)
            time_x = time_x.to(self.device, non_blocking=True)
            
            with autocast(enabled=self.amp):
                preds = self.model(x, time_x)
                loss = vectorized_quantile_loss(preds, y)
            
            total_loss += loss.item() * x.size(0)
        
        return total_loss / len(dataloader.dataset)

# Пример конфигурации
config = {
    'data_paths': ['data/*.csv'],
    'seq_len': 30,
    'pred_len': 5,
    'batch_size': 256,
    'num_workers': 8,
    'lr': 3e-4,
    'weight_decay': 1e-5,
    'epochs': 100,
    'patience': 7,
    'mixed_precision': True
}

In [3]:
# 1. Определение класса Dataset
class TimeSeriesDataset(Dataset):
    def __init__(self, data, time_features, seq_len=30, pred_len=5):
        self.data = data
        self.time_features = time_features
        self.seq_len = seq_len
        self.pred_len = pred_len
        
    def __len__(self):
        return len(self.data) - self.seq_len - self.pred_len
    
    def __getitem__(self, idx):
        x = self.data[idx:idx+self.seq_len]
        y = self.data[idx+self.seq_len:idx+self.seq_len+self.pred_len]
        
        time_x = self.time_features[idx:idx+self.seq_len]
        return torch.FloatTensor(x), torch.FloatTensor(y), torch.FloatTensor(time_x)


In [4]:
# 2. Функция Quantile Loss
def quantile_loss(preds, target, quantiles=[0.1, 0.3, 0.5, 0.7, 0.9]):
    """
    Многомерная версия (для всех признаков)
    Args:
        preds: [batch, pred_len, num_quantiles, features]
        target: [batch, pred_len, features]
    """
    target = target.unsqueeze(2)  # [batch, pred_len, 1, features]
    
    losses = []
    for i, q in enumerate(quantiles):
        errors = target - preds[..., i, :]  # [batch, pred_len, features]
        loss = torch.max((q-1)*errors, q*errors)
        losses.append(loss)
    
    return torch.mean(torch.cat(losses, dim=2))

In [5]:
# 4. Загрузка данных (пример)
def load_data():
    # Загрузка своих данных
    # data = ...  # [N, features]
    # time_data = ...  # [N, 7] в формате [год, месяц, день, час, минута, секунда, день_недели]
    # return train_data, val_data, train_time, val_time
    
    # Пример случайных данных
    data = np.random.randn(1000, 5)
    time_data = np.random.randint(2020, 2030, (1000, 7))
    return data[:800], data[800:], time_data[:800], time_data[800:]

train_data, val_data, train_time, val_time = load_data()

In [6]:
# 3. Инициализация модели и устройств
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PricePredictorModel().to(device)

In [7]:

# Нормализация данных
scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
val_data = scaler.transform(val_data)

# Создание DataLoader
train_dataset = TimeSeriesDataset(train_data, train_time)
val_dataset = TimeSeriesDataset(val_data, val_time)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# 5. Настройка оптимизатора и планировщика
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)

# 6. Цикл обучения
best_val_loss = float('inf')
patience = 5
counter = 0

In [8]:

for epoch in range(100):
    # Обучение
    model.train()
    train_loss = 0
    for x, y, time_x in train_loader:
        x = x.to(device)
        y = y.to(device)
        time_x = time_x.to(device)
        
        optimizer.zero_grad()
        
        preds = model(x, time_x)
        loss = quantile_loss(preds, y)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        train_loss += loss.item() * x.size(0)
    
    # Валидация
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x, y, time_x in val_loader:
            x = x.to(device)
            y = y.to(device)
            time_x = time_x.to(device)
            
            preds = model(x, time_x)
            loss = quantile_loss(preds, y)
            val_loss += loss.item() * x.size(0)
    
    # Расчет средних потерь
    train_loss /= len(train_loader.dataset)
    val_loss /= len(val_loader.dataset)
    
    scheduler.step(val_loss)
    
    # Логирование
    print(f"Epoch {epoch+1:03d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    
    # Early Stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth")
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered")
            break


Epoch 001 | Train Loss: 0.3453 | Val Loss: 0.2966
Epoch 002 | Train Loss: 0.3021 | Val Loss: 0.2854
Epoch 003 | Train Loss: 0.2989 | Val Loss: 0.2849
Epoch 004 | Train Loss: 0.2989 | Val Loss: 0.2839
Epoch 005 | Train Loss: 0.2975 | Val Loss: 0.2833
Epoch 006 | Train Loss: 0.2976 | Val Loss: 0.2832
Epoch 007 | Train Loss: 0.2973 | Val Loss: 0.2836
Epoch 008 | Train Loss: 0.2973 | Val Loss: 0.2832
Epoch 009 | Train Loss: 0.2969 | Val Loss: 0.2840


KeyboardInterrupt: 

In [ ]:

# 7. Загрузка лучшей модели
model.load_state_dict(torch.load("best_model.pth"))